In [2]:
import re
import pandas as pd

In [3]:
property_files = '../data/raw/property_meta'

In [4]:
df = pd.read_csv(f'{property_files}/property_metadata.csv')

In [5]:
df.head()

,url,name,type,cost_text,coordinates,rooms,desc
0,https://www.domain.com.au/5-1-3-rex-avenue-alp...,5/1-3 Rex Avenue Alphington VIC 3078,Apartment / Unit / Flat,$440.00,"[-37.7856068, 145.0249141]","[['2 Beds'], ['1 Bath'], ['1 Parking']]","class=""css-dxogle"">* Unverified feature<svg a..."
1,https://www.domain.com.au/805-81-queens-road-m...,805/81 Queens Road Melbourne VIC 3004,Studio,$380 weekly,"[-37.853804, 144.9798785]","[['0 Beds'], ['1 Bath'], ['1 Parking']]",Leasing Concierge
2,https://www.domain.com.au/12-1-rex-avenue-alph...,12/1 Rex Avenue Alphington VIC 3078,Apartment / Unit / Flat,$460.00,"[-37.7856092, 145.0249185]","[['2 Beds'], ['1 Bath'], ['1 Parking']]",Pyper Byrne
3,https://www.domain.com.au/3-lime-ct-bellfield-...,3 Lime Ct Bellfield VIC 3081,House,$425 PW | Weston Real Estate,"[-37.7548292, 145.041952]","[['3 Beds'], ['1 Bath'], ['3 Parking']]","class=""css-dxogle"">* Unverified feature<svg a..."
4,https://www.domain.com.au/24-36-ridley-street-...,24/36 Ridley Street Albion VIC 3020,Apartment / Unit / Flat,$250,"[-37.7834829, 144.8233623]","[['1 Bed'], ['1 Bath'], ['1 Parking']]","class=""css-dxogle"">* Unverified feature<svg a..."


In [6]:
df['postcode'] = df['name'].str[-4:]
df['name'] = df['name'].str[:-5]

In [7]:
rooms = df['rooms'].str.split(',', expand=True)
rooms = rooms.iloc[:, :-1]
rooms.columns = ['bed', 'bath', 'car']
rooms.fillna(0)
rooms['bed'] = rooms.bed.str.extract('(\d+)').fillna(0).astype('int64')
rooms['bath'] = rooms.bath.str.extract('(\d+)').fillna(0).astype('int64')
rooms['car'] = rooms.car.str.extract('(\d+)').fillna(0).astype('int64')

In [8]:
rooms.head()

,bed,bath,car
0,2,1,1
1,0,1,1
2,2,1,1
3,3,1,3
4,1,1,1


In [9]:
df = pd.concat([df, rooms], axis=1)
df = df.drop(columns=['rooms'])

In [10]:
df.head()

,url,name,type,cost_text,coordinates,desc,postcode,bed,bath,car
0,https://www.domain.com.au/5-1-3-rex-avenue-alp...,5/1-3 Rex Avenue Alphington VIC,Apartment / Unit / Flat,$440.00,"[-37.7856068, 145.0249141]","class=""css-dxogle"">* Unverified feature<svg a...",3078,2,1,1
1,https://www.domain.com.au/805-81-queens-road-m...,805/81 Queens Road Melbourne VIC,Studio,$380 weekly,"[-37.853804, 144.9798785]",Leasing Concierge,3004,0,1,1
2,https://www.domain.com.au/12-1-rex-avenue-alph...,12/1 Rex Avenue Alphington VIC,Apartment / Unit / Flat,$460.00,"[-37.7856092, 145.0249185]",Pyper Byrne,3078,2,1,1
3,https://www.domain.com.au/3-lime-ct-bellfield-...,3 Lime Ct Bellfield VIC,House,$425 PW | Weston Real Estate,"[-37.7548292, 145.041952]","class=""css-dxogle"">* Unverified feature<svg a...",3081,3,1,3
4,https://www.domain.com.au/24-36-ridley-street-...,24/36 Ridley Street Albion VIC,Apartment / Unit / Flat,$250,"[-37.7834829, 144.8233623]","class=""css-dxogle"">* Unverified feature<svg a...",3020,1,1,1


In [11]:
from collections import Counter

In [12]:
# Counter(" ".join(df["desc"].str.lower()).split()).most_common(100)

TypeError: sequence item 4207: expected str instance, float found

In [ ]:
# Counter(" ".join(df["desc"]).split()).most_common(100)

In [14]:
df.dtypes

url            object
name           object
type           object
cost_text      object
coordinates    object
desc           object
postcode       object
bed             int64
bath            int64
car             int64
dtype: object

In [15]:
df['postcode'].value_counts()

3000    223
3141    138
3182    123
3121    115
3181    110
       ... 
3913      1
3713      1
3090      1
3804      1
3036      1
Name: postcode, Length: 189, dtype: int64

In [16]:
df['type'].value_counts()

Apartment / Unit / Flat          2560
House                            2194
Townhouse                         581
Studio                            154
Villa                              16
Carspace                            8
Semi-Detached                       5
Acreage / Semi-Rural                5
Terrace                             4
Rural                               1
Penthouse                           1
New Apartments / Off the Plan       1
Duplex                              1
Name: type, dtype: int64

In [17]:
postcode_df = pd.read_csv('../data/raw/abs/australian_postcodes.csv')

In [18]:
postcode_df.head()

,id,postcode,locality,state,long,lat,dc,type,status,sa3,...,MMM_2015,MMM_2019,ced,altitude,chargezone,phn_code,phn_name,lgaregion,electorate,electoraterating
0,230,200,ANU,ACT,149.11900,-35.277700,NaN,NaN,NaN,NaN,...,1.0,1.0,NaN,NaN,N2,NaN,NaN,NaN,Durack,NaN
1,21820,200,Australian National University,ACT,149.11890,-35.277700,NaN,NaN,Added 19-Jan-2020,NaN,...,1.0,1.0,NaN,NaN,N2,NaN,NaN,NaN,Durack,NaN
2,232,800,DARWIN,NT,130.83668,-12.458684,NaN,NaN,Updated 6-Feb-2020,70101.0,...,2.0,2.0,NaN,NaN,NT1,PHN701,Northern Territory,Darwin,Solomon,Inner Metropolitan
3,24049,800,DARWIN CITY,NT,130.83668,-12.458684,NaN,NaN,Updated 6-Feb-2020,70101.0,...,2.0,2.0,NaN,NaN,NT1,PHN701,Northern Territory,Darwin,Solomon,Inner Metropolitan
4,233,801,DARWIN,NT,130.83668,-12.458684,NaN,NaN,Updated 25-Mar-2020 SA3,70101.0,...,2.0,2.0,NaN,NaN,NT1,PHN701,NaN,NaN,Lingiari,Rural


In [24]:
postcode_df = postcode_df[postcode_df['state'] == 'VIC']
columns = ['postcode', 'locality', 'long', 'lat', 'region', 'Lat_precise', 'Long_precise', 'SA2_MAINCODE_2016', 'SA2_NAME_2016', 'RA_2016', 'MMM_2019', 'altitude', 'phn_name', 'lgaregion']
postcode_df = postcode_df[columns]
postcode_df.head()

,postcode,locality,long,lat,region,Lat_precise,Long_precise,SA2_MAINCODE_2016,SA2_NAME_2016,RA_2016,MMM_2019,altitude,phn_name,lgaregion
6151,3000,MELBOURNE,144.956776,-37.817403,R1,-37.815207,144.963937,206041122.0,Melbourne,1.0,1.0,15.244751,North Western Melbourne,Melbourne
6152,3001,MELBOURNE,144.956776,-37.817403,R1,-37.813628,144.963058,206041122.0,Melbourne,1.0,1.0,15.244751,NaN,Moonee Valley
6153,3002,EAST MELBOURNE,144.982207,-37.818517,R1,-37.816144,144.980459,206041119.0,East Melbourne,1.0,1.0,14.315022,North Western Melbourne,Yarra
6154,3003,WEST MELBOURNE,144.949592,-37.810871,R1,-37.811450,144.925397,206041127.0,West Melbourne,1.0,1.0,15.316063,North Western Melbourne,Melbourne
6155,3004,MELBOURNE,144.970161,-37.844246,R1,-37.830158,144.980459,206041126.0,Southbank,1.0,1.0,-3492.000000,South Eastern Melbourne,Yarra


In [26]:
postcode_df['postcode'].value_counts().head()

3352    62
3401    48
3551    41
3691    37
3678    35
Name: postcode, dtype: int64